In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import scipy.stats as stats
import datetime

In [2]:
# 논산연암 데이터 불러오기
columns =['input_dt',  'feed_plan_cd','feed_plan_nm', 'farm_no', 'farm_pig_no',
       'feed_plan_pct', 'gyobae_dt', 'bunman_dt', 'wk_gubun', 'wk_dt','one_intake_kg', 'one_tm','one_kg',
       'one_now_kg', 'two_tm', 'two_kg', 'two_now_kg', 'three_tm','three_kg', 'three_now_kg', 'four_tm',
       'four_kg', 'four_now_kg','five_tm', 'five_kg', 'five_now_kg', 'warn_kg', 'min_kg', 'log_dt']
data_columns = columns + ['day_now_kg', 'B_now_kg', 'B_kg', 'B_per_now_kg','now_eudusu','now_eukg','next_bunman_sancha','next_jae','next_chongsan','next_silsan','next_eudusu']
data = pd.DataFrame(columns=data_columns)
non_pi = pd.read_csv('./csvs/comars/pigplan_1751.csv')
non_co = pd.read_csv('./csvs/comars/uit_feeding_hy_1751_논산연암.csv')
non_co = non_co[columns]
pig_list = non_pi['FARM_PIG_NO'].unique()
co_list = non_co['farm_pig_no'].unique()
for pig_id in tqdm(pig_list):
    if pig_id in co_list:
        tmp_co = non_co[non_co['farm_pig_no'] == pig_id]
        tmp_pi = non_pi[non_pi['FARM_PIG_NO'] == pig_id]
        gyobaes = list(tmp_pi['GYOBAE_DT'].unique())
        gyobaes.sort()
        gyobaes_list = []
        for i in gyobaes:
            tmp_gyo = str(i)
            gyo_1 = tmp_gyo[:4]
            gyo_2 = tmp_gyo[4:6]
            gyo_3 = tmp_gyo[6:]
            gyobaes_list.append(gyo_1 + '-' + gyo_2 + '-' +gyo_3)
        for gyobae_num in range(len(gyobaes_list)):
            tmp = tmp_co[tmp_co['gyobae_dt'] == gyobaes_list[gyobae_num]]
            if tmp.empty:
                pass
            else:
                tmp = tmp[tmp['wk_gubun'] == 'B']
                now_eudusu = tmp_pi[tmp_pi['GYOBAE_DT'] == gyobaes[gyobae_num]]['EUDUSU'].values[0]
                now_eukg = tmp_pi[tmp_pi['GYOBAE_DT'] == gyobaes[gyobae_num]]['EUKG'].values[0]
                if len(gyobaes_list) > gyobae_num+1:
                    silsan = tmp_pi[tmp_pi['GYOBAE_DT'] == gyobaes[gyobae_num+1]]['SILSAN'].values[0]
                    chongsan = tmp_pi[tmp_pi['GYOBAE_DT'] == gyobaes[gyobae_num+1]]['CHONGSAN'].values[0]
                    eudusu = tmp_pi[tmp_pi['GYOBAE_DT'] == gyobaes[gyobae_num+1]]['EUDUSU'].values[0]
                    sancha = tmp_pi[tmp_pi['GYOBAE_DT'] == gyobaes[gyobae_num+1]]['BUNMAN_SANCHA'].values[0]
                    jae = tmp_pi[tmp_pi['GYOBAE_DT'] == gyobaes[gyobae_num+1]]['JAE'].values[0]
                else:
                    sancha = None
                    if sancha == None:
                        silsan = None
                        chongsan = None
                        eudusu = None
                        jae = None
                tmp['now_eudusu'] = now_eudusu
                tmp['now_eukg'] = now_eukg
                tmp['next_bunman_sancha'] = sancha
                tmp['next_silsan'] = silsan
                tmp['next_chongsan'] = chongsan
                tmp['next_eudusu'] = eudusu
                tmp['next_jae'] = jae
                tmp['day_now_kg'] = tmp['one_now_kg']+tmp['two_now_kg']+tmp['three_now_kg']+tmp['four_now_kg']+tmp['five_now_kg']
                tmp['day_kg'] = tmp['one_kg']+tmp['two_kg']+tmp['three_kg']+tmp['four_kg']+tmp['five_kg']
                B_now_kg = 0
                for z in tmp['day_now_kg']:
                    B_now_kg += z
                B_kg = 0
                for z in tmp['day_kg']:
                    B_kg +=z
                if B_now_kg == 0 or Counter(tmp['day_now_kg'])[0] >= 3:
                    continue
                B_per_now_kg = (B_now_kg/B_kg)*100
                tmp['B_now_kg'] = B_now_kg
                tmp['B_kg'] = B_kg
                tmp['B_per_now_kg'] = B_per_now_kg
                data = pd.concat([data,tmp], axis = 0)


100%|██████████| 1751/1751 [00:15<00:00, 111.64it/s]
